In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/UMich/SI670/Final_Proj')

Mounted at /content/drive


In [ ]:
!ls

 calorie_regression_baseline.ipynb
 calorie_regression_basic_densenet121.ipynb
 calorie_regression_multiin_101_multiout.ipynb
 calorie_regression_multiin_emb_multiout_densenet121.ipynb
 calorie_regression_multiin_multiout_densenet121.ipynb
 calorie_regression_multiout_densenet121.ipynb
 clean_recipe_data_nutrition.csv
'EfficientNetV2 calorie_regression_multiin_101_multiout.ipynb'
 file.txt
 food-101
 food101_id2label.json
 food101_label2id.json
 food101_nutritionix.csv
 food-101.tar.gz
 get_recipe_images.ipynb
 huggingface_food101_swin.ipynb
 initial_multiin_101_multiout_densenet.h5
 initial_multiin_101_multiout_efficientnetb0.h5
 initial_multiin_101_multiout_mobilenet.h5
 initial_multiin_101_multiout_NASNetMobile.h5
 initial_multiin_multiout_densenet_2.h5
 initial_multiin_multiout_densenet.h5
'mobileNetV2 calorie_regression_multiin_101_multiout.ipynb'
 My_suffering.ipynb
'NASNetMobile calorie_regression_multiin_101_multiout.ipynb'
 no_cat_calorie_regression_multiin_101_multiout.ipynb


In [ ]:
import pandas as pd
from tqdm import tqdm
import requests
import cv2
import numpy as np
import tensorflow as tf
from keras.applications.densenet import DenseNet121, preprocess_input
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image
import zipfile
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
SEED = 670

# Load Data and Preprocess

Basic preprocessing, remove unneeded columns, replace NAs with UNK

In [ ]:
data = pd.read_csv('clean_recipe_data_nutrition.csv')
data = data[
    ['id', 'servings', 'calories_per_serving', 'protein_per_serving', 
     'fat_per_serving', 'carb_per_serving', 'meal_type', 'region', 'subregion']
]
data['meal_type'] = data['meal_type'].fillna(value='UNK_meal_type')
data['subregion'] = data['subregion'].fillna(value='UNK_subregion') 
data.head()

,id,servings,calories_per_serving,protein_per_serving,fat_per_serving,carb_per_serving,meal_type,region,subregion
0,1,2,388,17,22,29,main,latin_american,mexican
1,3,10,795,16,39,96,dessert,latin_american,mexican
2,4,12,384,17,25,25,main,latin_american,mexican
3,8,8,381,34,22,9,UNK_meal_type,latin_american,carribean
4,9,12,427,18,12,63,main,latin_american,mexican


# Load images, split data into train and test

In [ ]:
images = []
imgzip = zipfile.ZipFile("Recipe_Images_224_x_224.zip")
for f in tqdm(imgzip.infolist()):
    ifile = imgzip.open(f)
    img = np.asarray(Image.open(ifile))
    images.append(img)


100%|██████████| 8573/8573 [00:16<00:00, 504.67it/s]


In [ ]:
train_df, test_df = train_test_split(data, test_size = 0.2, random_state = SEED)

In [ ]:
train_df.head()

,id,servings,calories_per_serving,protein_per_serving,fat_per_serving,carb_per_serving,meal_type,region,subregion
4971,8160,4,127,4,9,9,salad,european,italian
2573,4249,6,300,6,12,44,side,asian,indian
786,1354,8,395,11,17,48,UNK_meal_type,european,"scandi,vian"
3684,6038,25,318,26,6,41,UNK_meal_type,european,"scandi,vian"
435,784,35,141,7,8,11,UNK_meal_type,latin_american,mexican


In [ ]:
train_idx = train_df['id'].values
test_idx = test_df['id'].values
train_images = np.array([images[idx] for idx in train_idx])
test_images = np.array([images[idx] for idx in test_idx])

In [ ]:
train_y = np.column_stack(
    (train_df['calories_per_serving'].values,
     train_df['protein_per_serving'].values,
     train_df['fat_per_serving'].values,
     train_df['carb_per_serving'].values)
)
test_y = np.column_stack(
    (test_df['calories_per_serving'].values,
     test_df['protein_per_serving'].values,
     test_df['fat_per_serving'].values,
     test_df['carb_per_serving'].values)
)

# Create model

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
IMG_SHAPE = (224, 224, 3)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
base_model.trainable = False

29084464/29084464 [==============================] - 2s 0us/step


In [ ]:
inputs = tf.keras.Input(shape=(224, 224, 3))
augmented_inputs = data_augmentation(inputs)
processed_out = preprocess_input(augmented_inputs)
base_out = base_model(processed_out, training = False)
flatten_out = GlobalAveragePooling2D()(base_out)
pool_out = Dropout(0.2)(flatten_out)
outputs = Dense(4, activation='linear')(pool_out)
model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = 'huber_loss',
    metrics = ['mean_absolute_error']
)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        703750

In [ ]:
es = EarlyStopping(monitor='val_mean_absolute_error', patience=3, restore_best_weights=True)
initial_epochs = 100
history = model.fit(
    x = train_images,
    y = train_y,
    batch_size = 32,
    epochs = initial_epochs,
    validation_split = 0.2,
    callbacks = [es]
)

Epoch 1/100
105/105 [==============================] - 32s 164ms/step - loss: 91.7621 - mean_absolute_error: 92.2542 - val_loss: 84.8605 - val_mean_absolute_error: 85.3531
Epoch 2/100
105/105 [==============================] - 14s 129ms/step - loss: 80.3436 - mean_absolute_error: 80.8359 - val_loss: 75.8737 - val_mean_absolute_error: 76.3659
Epoch 3/100
105/105 [==============================] - 13s 128ms/step - loss: 71.8100 - mean_absolute_error: 72.3025 - val_loss: 68.7209 - val_mean_absolute_error: 69.2127
Epoch 4/100
105/105 [==============================] - 14s 131ms/step - loss: 65.2314 - mean_absolute_error: 65.7243 - val_loss: 63.4127 - val_mean_absolute_error: 63.9059
Epoch 5/100
105/105 [==============================] - 14s 132ms/step - loss: 60.3288 - mean_absolute_error: 60.8207 - val_loss: 59.4502 - val_mean_absolute_error: 59.9434
Epoch 6/100
105/105 [==============================] - 14s 129ms/step - loss: 56.8413 - mean_absolute_error: 57.3345 - val_loss: 56.5604 - v

In [ ]:
test_preds = model.predict(test_images)

33/33 [==============================] - 6s 127ms/step


In [ ]:
test_preds

array([[273.0436  ,   6.041163,  12.67064 ,  27.732761],
       [330.0619  ,   7.531645,  13.069391,  28.989126],
       [283.30585 ,   7.658527,  14.356565,  29.705616],
       ...,
       [304.47052 ,   9.688883,  14.967466,  29.974384],
       [305.98422 ,   6.26648 ,  14.150915,  27.175156],
       [318.96164 ,   8.409993,  15.062287,  31.077328]], dtype=float32)

In [ ]:
mae_cal = mean_absolute_error(test_y[:,0:1], test_preds[:,0:1])
mae_pro = mean_absolute_error(test_y[:,1:2], test_preds[:,1:2])
mae_fat = mean_absolute_error(test_y[:,2:3], test_preds[:,2:3])
mae_carb = mean_absolute_error(test_y[:,3:4], test_preds[:,3:4])

print(mae_cal, mae_pro, mae_fat, mae_carb)

170.63712663029347 12.059716353128696 10.675056293092926 18.188449223836262
